<a href="https://colab.research.google.com/github/Kiranchawla09/Reddit-Flair-Detection/blob/master/Reddit_Flair_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing the Libraries

In [1]:
import pandas as pd
import datetime as dt
!pip install praw
import nltk
nltk.download('stopwords')
import re
import praw as pr                   #module for reddit
from bs4 import BeautifulSoup       #Access the HTML of the webpage and extract useful information/data from it.
                                    #It is of ‘string’ type.

     |████████████████████████████████| 133kB 43.7MB/s 
     |████████████████████████████████| 204kB 46.4MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Loading the Application details

In [14]:
reddit = pr.Reddit(client_id='EuOy7dnVRlVDew', 
                     client_secret='SGDTw27oQfUc3i7S61rJhPknpgA',
                     user_agent='reddit flair',
                     username='kiran5_chawla', 
                     password='/Haathikhai')

print(reddit.read_only)  
reddit.read_only = True

False


#### India Reddit Data

In [0]:
subreddit = reddit.subreddit('india')

In [0]:
flairs = ["AskIndia", "Non-Political", "Policy/Economy","AMA", "Politics","[R]eddiquette", 
          "Photography","Business/Finance","Science/Technology","Sports", "Scheduled","Food"]

my_dict = { "Flair":[],
             "Title":[],
             "Created_at":[],
             "Score":[],
             "Url_address":[],
             "Comments":[],
             "#Comments":[],
             "Body":[],
             "Author":[]
           }

## Collecting the Data

In [0]:
for f in flairs:
    collect_subreddit= subreddit.search (f,limit=1000)
    #collecting 1000 data for each flair 
    for i in collect_subreddit:
        
        my_dict["Flair"].append (f)
        my_dict["Title"].append (i.title)
        my_dict["Created_at"].append (i.created)
        my_dict["Score"].append (i.score)
        my_dict["Url_address"].append(i.url)
        my_dict["#Comments"].append (i.num_comments)
        my_dict["Body"].append(i.selftext)
        my_dict["Author"].append (i.author)

        i.comments.replace_more(limit=None)
        comment = ''
        for top_level_comment in i.comments:
               comment = comment + ' ' + top_level_comment.body
        my_dict["Comments"].append(comment)

In [0]:
my_data = pd.DataFrame(my_dict)

In [0]:
#my_data

In [0]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

def word_count (text):
    return 
    
words = my_data["Body"].apply(lambda x: len(str(x).split(" ")))   
my_data= my_data.assign (Word_Count= words)      #no. of words in the body


chars= my_data["Body"].str.len()
my_data= my_data.assign (Char_Count= chars)

hashs= my_data["Body"].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))   
my_data= my_data.assign (Hashtags= hashs) #no. of hashtags in the body

_timestamp = my_data["Created_at"].apply(get_date)
my_data = my_data.assign(timestamp = _timestamp)

## Data Cleaning

In [0]:
from nltk.corpus import stopwords
Replace_by_space = re.compile('[/(){}\[\]\|@,;]')
Replace_bad_symbols = re.compile('[^0-9a-z #+_]')
Stopwords = set(stopwords.words('english'))

def clean_data(text):
    text = text.lower()
    text = Replace_by_space.sub(' ', text)
    text = Replace_bad_symbols.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in Stopwords)
    return text

def string_form(value):
    return str(value)
   
def rare_words(text):
    text= text.split()
    freq = pd.Series(' '.join(my_data['Body']).split()).value_counts()[-10:]
    freq = list(freq.index)
    text = " ".join(word for word in text if word not in freq)
    #print (text)
    return text

my_data['Title'] = my_data['Title'].apply(string_form)
my_data['Body'] = my_data['Body'].apply(string_form)
my_data['Comments']=my_data['Comments'].apply(string_form)

my_data['Title'] = my_data['Title'].apply(clean_data)
my_data['Body'] = my_data['Body'].apply(clean_data)
my_data['Comments']=my_data['Comments'].apply(clean_data)

my_data['Body']= my_data['Body'].apply(rare_words)
my_data['Comments']=my_data['Comments'].apply(rare_words)

#### Saving to csv


In [0]:
my_data.to_csv('data_scraped4.csv', index=False)

In [0]:
# Importing the dataset
dataset = pd.read_csv('data_scraped4.csv')
X= dataset.iloc [:, [1,2,3,4,5,6,7,8,9,10,11,12]]
y= dataset.iloc [:,0]

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)